# Data Analysis - Messaging Agent Performance

This notebook analyzes conversation data, performance metrics, and user interactions with the messaging agent.

## What you'll analyze:
- Conversation patterns and flows
- Response times and performance metrics
- User satisfaction and engagement
- Error rates and failure patterns
- Tool usage and effectiveness


In [ ]:
# Import libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import json
from typing import Dict, List, Any

# Google Cloud imports
from google.cloud import bigquery
from google.cloud import storage

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("📊 Data analysis libraries loaded successfully!")


In [ ]:
# Configuration
PROJECT_ID = os.getenv('PROJECT_ID', 'your-project-id')
DATASET_ID = 'messaging_agent'
TABLE_ID = 'conversations'

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"🔍 BigQuery client initialized for project: {PROJECT_ID}")


In [ ]:
# Load conversation data from BigQuery
def load_conversation_data(days_back=30):
    """Load conversation data from the last N days"""
    
    query = f"""
    SELECT 
        timestamp,
        user_id,
        session_id,
        message,
        response,
        response_time_ms,
        tools_used,
        rag_used,
        error_message,
        satisfaction_score
    FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`
    WHERE timestamp >= TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL {days_back} DAY)
    ORDER BY timestamp DESC
    """
    
    df = client.query(query).to_dataframe()
    return df

# Load data
df = load_conversation_data(30)
print(f"📈 Loaded {len(df)} conversations from the last 30 days")
print(f"📅 Date range: {df['timestamp'].min()} to {df['timestamp'].max()}")


In [ ]:
# Basic data exploration
print("📊 Dataset Overview:")
print(f"Total conversations: {len(df)}")
print(f"Unique users: {df['user_id'].nunique()}")
print(f"Unique sessions: {df['session_id'].nunique()}")
print(f"Average response time: {df['response_time_ms'].mean():.2f} ms")
print(f"Error rate: {(df['error_message'].notna().sum() / len(df) * 100):.2f}%")

print("\n🔍 First few rows:")
df.head()


In [ ]:
# Response time analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Response time distribution
axes[0, 0].hist(df['response_time_ms'], bins=50, alpha=0.7, color='skyblue')
axes[0, 0].set_title('Response Time Distribution')
axes[0, 0].set_xlabel('Response Time (ms)')
axes[0, 0].set_ylabel('Frequency')

# Response time over time
df['date'] = df['timestamp'].dt.date
daily_avg = df.groupby('date')['response_time_ms'].mean()
axes[0, 1].plot(daily_avg.index, daily_avg.values, marker='o')
axes[0, 1].set_title('Average Response Time Over Time')
axes[0, 1].set_xlabel('Date')
axes[0, 1].set_ylabel('Average Response Time (ms)')
axes[0, 1].tick_params(axis='x', rotation=45)

# Response time by hour
df['hour'] = df['timestamp'].dt.hour
hourly_avg = df.groupby('hour')['response_time_ms'].mean()
axes[1, 0].bar(hourly_avg.index, hourly_avg.values, color='lightcoral')
axes[1, 0].set_title('Average Response Time by Hour')
axes[1, 0].set_xlabel('Hour of Day')
axes[1, 0].set_ylabel('Average Response Time (ms)')

# Response time box plot
df.boxplot(column='response_time_ms', ax=axes[1, 1])
axes[1, 1].set_title('Response Time Distribution (Box Plot)')
axes[1, 1].set_ylabel('Response Time (ms)')

plt.tight_layout()
plt.show()
